# Detección de objetos en imágenes con CNNs: Conteo de canastas de baloncesto 🏀 

El objetivo principal de este proyecto es desarrollar un sistema de visión por computadora capaz de detectar el balón y el aro de baloncesto en un video, utilizando una red neuronal convolucional basada en la arquitectura YOLOv8. A partir de estas detecciones, el sistema implementa una lógica para contar automáticamente la cantidad de canastas realizadas durante el transcurso del video. Esta aplicación puede ser útil en contextos deportivos para análisis automático de jugadas o recopilación de estadísticas.

## 1. Dataset description, download and exploration

- **Nombre del conjunto de datos**: Basketball Detection (Versión 1)
- **Fuente**: Roboflow Universe  
  - **URL**: [https://universe.roboflow.com/basketball-6vyfz/basketball-detection-srfkd](https://universe.roboflow.com/basketball-6vyfz/basketball-detection-srfkd)
  - **Licencia**: CC BY 4.0

- **Contenido**: El conjunto de datos contiene imágenes de partidos de baloncesto anotadas con *bounding boxes* para distintos objetos clave en el juego.

- **Clases de objetos**:
  - `0`: Ball  
  - `1`: Hoop with ball 
  - `2`: Player  
  - `3`: Hoop  
  - `4`: Player shooting  

- **Formato de anotación**: YOLOv5/YOLOv8. Cada archivo `.txt` contiene una línea por objeto detectado con el siguiente formato:
`<class_id> <x_center> <y_center> <width> <height>`.

- **Distribución del dataset**:
  - **Entrenamiento**: 9,599 imágenes (88%)
  - **Validación**: 873 imágenes (8%)
  - **Pruebas**: 436 imágenes (4%)

## 2. Data preparation and augmentation

Se aplicaron augmentations automáticas provistas por YOLOv8, incluyendo:
- Escalado aleatorio
- Desenfoque (blur)
- CLAHE (mejora de contraste)
- Transformación a escala de grises

Esto ayudó a mejorar la robustez del modelo y evitar el sobreajuste.

## 3. Model selection

Se seleccionó el modelo `yolov8l.pt` (versión large de YOLOv8) como punto de partida por su buen balance entre velocidad y precisión. El modelo fue ajustado para detectar las 5 clases del dataset.

## 4. Training

El modelo fue entrenado durante **10 épocas** con imágenes de tamaño **640x640**. Se utilizó el optimizador **AdamW**, con validación automática después de cada época. El modelo mostró mejoras constantes en precisión y recall.

- **Código de entrenamiento utilizado**:

Instalar la librería ultralytics

In [ ]:
!pip install ultralytics

from google.colab import drive, files
import shutil
from ultralytics import YOLO
import os

Montar Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Cargar modelo YOLOv8l

In [ ]:
# Cargar el modelo
def load_model(model_path="yolov8l.pt"):
    model = YOLO(model_path)
    return model

Entrenar el modelo con el dataset

In [ ]:
# Función para reentrenar el modelo y guardarlo
def retrain_model(model, train_data, epochs=10, batch=16, save_path="yolov8_custom.pt"):
    # Entrenar el modelo con los datos proporcionados
    model.train(data=train_data, epochs=epochs, batch=batch)

    # Guardar el modelo reentrenado
    model.save(save_path)
    print(f"Modelo guardado en {save_path}")

Main

In [ ]:
# Cargar el modelo
model_path = "yolov8l.pt"
model = load_model(model_path)

# Ruta del archivo de datos (data.yaml) que contiene las rutas de las imágenes de entrenamiento y validación
train_data = "/content/drive/MyDrive/basketball/data.yaml"

# Ruta de guardado del modelo entrenado
save_path = "/content/drive/MyDrive/yolov8_custom.pt"

# Reentrenar el modelo
retrain_model(model, train_data, save_path=save_path)

100%|██████████| 83.7M/83.7M [00:00<00:00, 104MB/s] 


Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/basketball/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 755k/755k [00:00<00:00, 21.1MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 96.8MB/s]


...

      10/10      11.2G      1.021      0.682      1.226         34        640: 100%|██████████| 600/600 [07:37<00:00,  1.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:17<00:00,  1.56it/s]

                   all        873       3408      0.863       0.84      0.902      0.649



10 epochs completed in 1.859 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 87.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 87.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 43,610,463 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:20<00:00,  1.34it/s]


                   all        873       3408      0.865       0.84      0.902      0.649
                     0        751        876      0.897      0.732      0.845      0.585
                     1         78         78      0.837      0.858       0.91        0.7
                     2        549       1690      0.878      0.875      0.923      0.637
                     3        573        600      0.966      0.914      0.965      0.732
                     4        164        164      0.745      0.823      0.865      0.593
Speed: 0.3ms preprocess, 15.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train
Modelo guardado en /content/drive/MyDrive/yolov8_custom.pt


## 5. Evaluation on the test set

Se evaluó el modelo final (`best.pt`) obteniendo los siguientes resultados promedio:
- **Precisión (P):** 0.865  
- **Recall (R):** 0.840  
- **mAP@0.5:** 0.902  
- **mAP@0.5:0.95:** 0.649  

**Precisión por clase:**
- **Clase 0:** 0.865  
- **Clase 1:** 0.878  
- **Clase 3:** 0.966  

Las métricas indican un buen rendimiento general en todas las clases.

## 6. Results and discussion

El modelo logró una alta precisión en la mayoría de las clases, especialmente en la clase 3 (0.966 de precisión y 0.914 de recall). También se observaron buenos resultados en la clase 1 (precisión de 0.878) y clase 0 (precisión de 0.865). Sin embargo, hay margen de mejora en clases con menor recall. Posibles siguientes pasos:
- Entrenar más épocas
- Mejorar la calidad de las etiquetas
- Ajustar el esquema de augmentations